[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). Instance produced by Yu Zhou Zhang. All rights reserved.

# My Covid-19 Dashboard

This is my Covid-19 Dashboard, which displays the figures for Covid tests, cases and deaths in England over time, both with raw data and proportionally. The dashboard is displayed using [voila](https://voila.readthedocs.io/en/stable/index.html), a Python dashboarding tool that converts notebooks to standalone dashboards. 

In [ ]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [ ]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

In [ ]:
# Loads a JSON file to store the raw data in jsondata
with open('testsAndCases.json', 'r') as fileinput:
    filedata = fileinput.read()
    jsondata=json.loads(filedata)

In [ ]:
# parsing date string into datetime
def parse_date(datestring):
    return pd.to_datetime(datestring, format="%Y-%m-%d")

# 2 options for wrangling the data, raw and proportional figures
def wrangle_data(rawdata, proportional=False):

    datalist = rawdata['data']
    
    # building a list of all dates that have records
    dates = [record['date'] for record in datalist]
    dates.sort()
    startDate = parse_date(dates[0])
    endDate = parse_date(dates[-1])
    # creating an index to be used in the dataframe
    index = pd.date_range(startDate, endDate, freq='D')
    
    myDataFrame = pd.DataFrame(index=index, columns=['tests', 'cases', 'deaths'])
    
    
    maximum_values = {}
    # saving the maximum value of each column of interest into a dictionary
    for dayData in datalist:
        for k, v in dayData.items():
            if k in ['tests', 'cases', 'deaths']:
                
                if v is not None:
                    
                    if v > maximum_values.get(k, 0):
                        maximum_values[k] = v
                
                        
    
    
    # iterating through datalist to populate the dataframe
    for record in datalist:
        date = parse_date(record['date'])
        for column in ['tests', 'cases', 'deaths']:
            
            if pd.isna(myDataFrame.loc[date, column]):
                value = float(record[column]) if record[column] != None else 0.0
                if proportional:
                    # divides the value by the maximum value stored in the dictionary
                    value /= maximum_values.get(column, 1)
                
                myDataFrame.loc[date, column] = value
    myDataFrame.fillna(0.0, inplace=True)
    
    
    return myDataFrame

# calling wrangle_data on the jsondata twice, supplying the raw and proportional dataframes:
df=wrangle_data(jsondata)# df is the dataframe for plotting
dfProportional=wrangle_data(jsondata, proportional=True)


In [ ]:
# API access code
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    filters = [
    'areaType=nation',
    'areaName=England'
]
    structure = {
    "date": "date",
    "tests": "newTestsByPublishDate",
    "cases": "newCasesByPublishDate",
    "deaths": "newDeaths28DaysByPublishDate"
}
    api = Cov19API(filters=filters, structure=structure)
    testsAndCases=api.get_json()
    return testsAndCases # return data read from the API

In [ ]:

def api_button_callback(button):
    # Getting fresh data from the API
    try:
        apidata=access_api()
        global df
        global dfProportional
        # wrangles the data and overwrites the 2 dataframes for plotting
        df=wrangle_data(apidata)
        dfProportional=wrangle_data(apidata, proportional=True)

        # Giving feedback to the user by changing the appearance of the button after accessing the API
        apibutton.icon="check"
        apibutton.disabled=True
    
  
    except:
        print('Sorry, it seems there is a problem fetching the data')
        # Changing appearance of API button in the even of an error
        apibutton.icon="unlink"
        apibutton.description='Unavailable'
        

# the button
apibutton=wdg.Button(
    description='Fetch Data',
    disabled=False,
    button_style='', 
    tooltip="Keep calm and carry on",
    icon='download'
)

# registering the button callback function with the button
apibutton.on_click(api_button_callback) 
display(apibutton)


# Comparing tests, cases and deaths

Below, you can select the metrics that you want to view (Ctrl + click to select multiple) and choose whether to view raw figures or figures scaled proportionally to its maximum value.
This proportional scaling allows you to visualise more clearly how each metric is influenced by others.

In [ ]:
%matplotlib inline

# make figures larger
plt.rcParams['figure.dpi'] = 100

# multiple select for columns
testsCasesDeathsColumns = wdg.SelectMultiple(
    options = ['tests', 'cases', 'deaths'],
    value = ['cases', 'deaths'],
    rows = 3,
    description = 'Parameters',
    disabled = False)

# radio button select for proportional vs raw data
proportion=wdg.RadioButtons(
    options=['raw', 'proportional'],
    value='proportional',
    layout={'width': 'max-content'},
    description='Scale:',
    disabled=False)

# grouping into a box
controls=wdg.HBox([testsCasesDeathsColumns, proportion])

# function to update graph
def testsCasesDeathsGraph(columns, proportion):
    
    numberOfColumns = len(columns)
    if numberOfColumns > 0:
        if proportion == 'proportional':
            dfProportional.plot(y = list(columns))
        else:
            df.plot(y = list(columns))
        plt.show()
    else:
        print('Click to select data for graph')

output = wdg.interactive_output(testsCasesDeathsGraph, {'columns': testsCasesDeathsColumns, 'proportion': proportion})

# grouping controls and graph into a box
form=wdg.HBox([output, controls])

# displaying graph
display(form)


## Acknowledgements

Created under the guidance of Prof. Fabrizio Smeraldi. Thanks for the support!

*Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*